In [4]:
import findspark
findspark.init()

import pyspark
import pandas as pd
import numpy as np
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.pandas import *

C:\Spark\spark-3.4.0-bin-hadoop3\python\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [5]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").getOrCreate()

In [6]:
df = spark.read.json(path= r'C:\Users\ADMIN\PycharmProjects\Bigdata-1\Data\20230126\data.json')

In [7]:
df.printSchema()

root
 |-- AppId: string (nullable = true)
 |-- Event: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- ItemName: string (nullable = true)
 |-- device: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_name: string (nullable = true)
 |-- isp: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- receive_time: string (nullable = true)



In [8]:
df.show(100)

+--------------------+------------+--------------------+--------------------+--------------+--------------------+-----------------+-------+--------------------+-------------------+
|               AppId|       Event|              ItemId|            ItemName|        device|           device_id|      device_name|    isp|            platform|       receive_time|
+--------------------+------------+--------------------+--------------------+--------------+--------------------+-----------------+-------+--------------------+-------------------+
|                IPTV|StartChannel|             vtv5-hd|             VTV5 HD|        BoxOTT|   D4:CF:F9:63:8C:23|        SEI500FPT|    fpt|fplay-ottbox-sei-...|2023-01-26T20:02:59|
|                IPTV|StartChannel|             vtv5-hd|             VTV5 HD|        BoxOTT|   D4:CF:F9:63:6B:C0|        SEI500FPT|   vnpt|fplay-ottbox-sei-...|2023-01-26T20:02:59|
|                IPTV|StartChannel|             vtv4-hd|             VTV4 HD|       SmartTV|   

In [9]:
# Top 10 kênh truyền hình được xem nhiều nhất
df[df['AppID'] == 'IPTV'].groupBy('ItemName').count().orderBy('count', ascending= False).show(10)

+--------------+------+
|      ItemName| count|
+--------------+------+
|       VTV1 HD|520679|
|       VTV3 HD|267147|
|Vĩnh Long 1 HD|179205|
|Vĩnh Long 3 HD|109937|
|       VTV2 HD|108524|
|Vĩnh Long 2 HD|103970|
|       VTV5 HD| 97282|
|   VTV Cần Thơ| 93693|
|       VTV4 HD| 77537|
|     Đồng Tháp| 74146|
+--------------+------+
only showing top 10 rows



In [10]:
# Top 10 content được search nhiều nhất
df[df['AppID'] == 'SEARCH'].groupBy('ItemName').count().orderBy('count', ascending= False).show(10)

+--------------------+-----+
|            ItemName|count|
+--------------------+-----+
|     Hướng Gió Mà Đi| 2474|
|    Vẻ Đẹp Đích Thực| 2474|
|     Tinh Hà Xán Lạn| 1804|
|Thử Thách Thần Tượng| 1545|
|Bạn Trai Tôi Là H...| 1483|
|Khanh Khanh Nhật ...| 1247|
|Kết Hôn Rồi Bắt Đ...| 1244|
|  Lưỡi Gươm Diệt Quỷ| 1147|
|Cuộc Chiến Thượng...|  965|
|         Hội Pháp Sư|  956|
+--------------------+-----+
only showing top 10 rows



In [11]:
# Top 10 content được sử dụng không thông qua IPTV
df[df['AppID'] != 'IPTV'].groupBy('ItemName').count().orderBy('count', ascending= False).show(10)

+--------------------+-----+
|            ItemName|count|
+--------------------+-----+
|     Hướng Gió Mà Đi|28867|
|Khanh Khanh Nhật ...|15879|
|    Hạ Cuối Tình Đầu|14484|
|     Tinh Hà Xán Lạn|12372|
|             VTV1 HD|11146|
|    Vẻ Đẹp Đích Thực| 9981|
|Buông Tay Đi Đồ N...| 8646|
|    Sứ Giả Thần Chết| 8411|
|Định Luật 80/20 C...| 8046|
|Học Viện Anh Hùng...| 7933|
+--------------------+-----+
only showing top 10 rows



In [12]:
df.groupBy('device').count().orderBy('count', ascending= False).show(10)

+--------------+-------+
|        device|  count|
+--------------+-------+
|        BoxOTT|1982586|
|SmartTvAndroid|1073534|
|       SmartTV| 837684|
|        Mobile| 354321|
|         WebOS|  42658|
+--------------+-------+



In [13]:
df.groupBy('ItemName').agg(countDistinct('device_id')).orderBy('count(device_id)', ascending= False).show(10)

+--------------+----------------+
|      ItemName|count(device_id)|
+--------------+----------------+
|       VTV1 HD|          179468|
|       VTV3 HD|           94095|
|Vĩnh Long 1 HD|           69167|
|       VTV2 HD|           44605|
|      FPT Play|           40862|
|       VTV5 HD|           40850|
|Vĩnh Long 3 HD|           39638|
|   VTV Cần Thơ|           39312|
|Vĩnh Long 2 HD|           38760|
|       VTV4 HD|           31885|
+--------------+----------------+
only showing top 10 rows



In [23]:
df.groupBy('ItemName').agg(count('device_id')).orderBy('count(device_id)', ascending= False).show(10)

+--------------+----------------+
|      ItemName|count(device_id)|
+--------------+----------------+
|       VTV1 HD|          531825|
|       VTV3 HD|          268745|
|Vĩnh Long 1 HD|          184151|
|Vĩnh Long 3 HD|          112323|
|       VTV2 HD|          108974|
|Vĩnh Long 2 HD|          106044|
|       VTV5 HD|           97625|
|   VTV Cần Thơ|           94002|
|       VTV4 HD|           77748|
|     Đồng Tháp|           75924|
+--------------+----------------+
only showing top 10 rows



In [14]:
df = df.withColumn("Hour", hour('receive_time'))
df = df.withColumn("Date", to_date('receive_time'))
df.show(10)

+------+------------+--------------------+--------------------+--------------+--------------------+------------+-------+--------------------+-------------------+----+----------+
| AppId|       Event|              ItemId|            ItemName|        device|           device_id| device_name|    isp|            platform|       receive_time|Hour|      Date|
+------+------------+--------------------+--------------------+--------------+--------------------+------------+-------+--------------------+-------------------+----+----------+
|  IPTV|StartChannel|             vtv5-hd|             VTV5 HD|        BoxOTT|   D4:CF:F9:63:8C:23|   SEI500FPT|    fpt|fplay-ottbox-sei-...|2023-01-26T20:02:59|  20|2023-01-26|
|  IPTV|StartChannel|             vtv5-hd|             VTV5 HD|        BoxOTT|   D4:CF:F9:63:6B:C0|   SEI500FPT|   vnpt|fplay-ottbox-sei-...|2023-01-26T20:02:59|  20|2023-01-26|
|  IPTV|StartChannel|             vtv4-hd|             VTV4 HD|       SmartTV|       F5AYANSVAEPIF|Samsung-202

In [15]:
time_usage_behaviour = df.groupBy('Hour').count().orderBy('count', ascending= False)
time_usage_behaviour.show(10)

+----+------+
|Hour| count|
+----+------+
|  20|444152|
|  19|397408|
|  18|359391|
|  21|348988|
|  17|262650|
|  16|224279|
|  11|220327|
|  10|216520|
|  12|198117|
|  15|192053|
+----+------+
only showing top 10 rows



In [21]:
test = df.groupBy(['Date', 'Hour', 'device', 'device_name', 'platform',  'AppID', 'ItemName']).agg(count('device_id').alias('Total_log'),
                                                                                                    countDistinct('device_id').alias('Total_devices'))

In [42]:
test1 = df.groupBy(['device', 'device_name', 'platform',  'AppID', 'ItemName']).agg(count('device_id').alias('Total_log'),
                                                                                    countDistinct('device_id').alias('Total_devices'),
                                                                                    first('Date').alias('Date'), 
                                                                                    first('Hour').alias('Hour'))

In [22]:
print((test.count(), len(test.columns)))

(629033, 9)


In [28]:
print((test1.count(), len(test1.columns)))

(226269, 9)


In [25]:
test.show(10)

+----------+----+--------------+--------------+--------------------+--------+--------------------+---------+-------------+
|      Date|Hour|        device|   device_name|            platform|   AppID|            ItemName|Total_log|Total_devices|
+----------+----+--------------+--------------+--------------------+--------+--------------------+---------+-------------+
|2023-01-26|  19|        BoxOTT|        FHRT2X|fplay-ottbox-hise...|    IPTV|     HTVC Thuần Việt|       70|           60|
|2023-01-26|  20|       SmartTV|  Samsung-2020|  smarttv-ss-nextgen|    IPTV|       K+ Sport 1 HD|      112|           87|
|2023-01-26|  20|        BoxOTT|     SEI500FPT|fplay-ottbox-sei-...|    IPTV|         Bạc Liêu HD|      228|          193|
|2023-01-26|  20|SmartTvAndroid|    BRAVIA_VU3|smarttv-sony-android|    IPTV|            Hà Giang|       17|           14|
|2023-01-26|  20|SmartTvAndroid| BRAVIA_UR3_PA|smarttv-sony-android|    IPTV|             VTV3 HD|      259|          171|
|2023-01-26|  18

In [37]:
test[(test['ItemName'] == 'VTV3 HD') & (test['device_name'] == 'BRAVIA_UR3_PA') & (test['Hour'] == 0)].show(100)

+----------+----+--------------+-------------+--------------------+-----+--------+---------+-------------+
|      Date|Hour|        device|  device_name|            platform|AppID|ItemName|Total_log|Total_devices|
+----------+----+--------------+-------------+--------------------+-----+--------+---------+-------------+
|2023-01-26|   0|SmartTvAndroid|BRAVIA_UR3_PA|smarttv-sony-android| IPTV| VTV3 HD|        7|            6|
+----------+----+--------------+-------------+--------------------+-----+--------+---------+-------------+



In [38]:
df[(df['ItemName'] == 'VTV3 HD') & (df['device_name'] == 'BRAVIA_UR3_PA') & (df['Hour'] == 0) ].show(100)

+-----+------------+-------+--------+--------------+----------------+-------------+-------+--------------------+-------------------+----+----------+
|AppId|       Event| ItemId|ItemName|        device|       device_id|  device_name|    isp|            platform|       receive_time|Hour|      Date|
+-----+------------+-------+--------+--------------+----------------+-------------+-------+--------------------+-------------------+----+----------+
| IPTV|StartChannel|vtv3-hd| VTV3 HD|SmartTvAndroid|e1e732beec9f1beb|BRAVIA_UR3_PA|viettel|smarttv-sony-android|2023-01-26T00:03:57|   0|2023-01-26|
| IPTV|StartChannel|vtv3-hd| VTV3 HD|SmartTvAndroid|974031629808f7c5|BRAVIA_UR3_PA|viettel|smarttv-sony-android|2023-01-26T00:08:01|   0|2023-01-26|
| IPTV|StartChannel|vtv3-hd| VTV3 HD|SmartTvAndroid|fcb8553b1345fcf1|BRAVIA_UR3_PA|    fpt|smarttv-sony-android|2023-01-26T00:20:10|   0|2023-01-26|
| IPTV|StartChannel|vtv3-hd| VTV3 HD|SmartTvAndroid|8e0807bd4d850033|BRAVIA_UR3_PA|   vnpt|smarttv-sony-an

In [43]:
test1[(test1['ItemName'] == 'VTV3 HD') & (test1['device_name'] == 'BRAVIA_UR3_PA') & (test1['Hour'] == 0)].show(10)


+------+-----------+--------+-----+--------+---------+-------------+----+----+
|device|device_name|platform|AppID|ItemName|Total_log|Total_devices|Date|Hour|
+------+-----------+--------+-----+--------+---------+-------------+----+----+
+------+-----------+--------+-----+--------+---------+-------------+----+----+

